In [1]:
import math
import numpy as np

# vypocet funkcni hodnoty f v x
def fx(p, x):
    px = []
    i = 0
    for t in p:
        px.append((t*(x**(len(p)-i-1))))
        i = i + 1
    return sum(px)

# derivace polynomu
def derivation(p):
    derivated=[]
    for i in range(len(p)-1):
        derivated.append((len(p)-i-1)*p[i])
    return derivated

# newtonova metoda pro nalezeni korenu polynomu
def newtons_method(p, x, n, delta): 
    xm1 = 0
    i = 0
    while (i < n):
        xm1 = x
        xi = x - (fx(p, x)/fx(derivation(p), x))
        x = xi
        i = i + 1
        if abs(x - xm1) < delta:
            break
    if isinstance(x, complex):
        cx = []
        cx.append(x)  # komplexni koren
        cx.append(-x) # komplexne sdruzeny koren
        return cx
    else:
        return round(x, 9) # realny koren

# deleni dvou polynomu
def divide_polynoms(p1, p2):
    q, r = np.polynomial.polynomial.polydiv(p1, p2)
    if r[0] == 0 and len(r) == 1:
        return q
    else:
        print('Není možné dělit polynomy beze zbytku.')
        return [0]

# nalezeni korenu kvadratickeho polynomu
def equation_roots(pq): 
    a = pq[0]
    b = pq[1]
    c = pq[2]
    roots = []
    
    diskriminant = b**2 - 4 * a * c 
    sqrt_val = math.sqrt(abs(diskriminant)) 
    
    # dva realne rozdilne koreny
    if diskriminant > 0: 
        roots.append((-b + sqrt_val)/(2 * a)) 
        roots.append((-b - sqrt_val)/(2 * a)) 
    
    # jeden realny koren
    elif diskriminant == 0: 
        roots.append(-b / (2 * a)) 
    
    # komplexni koreny
    else:
        roots.append(- b / (2 * a), " + i", sqrt_val) 
        roots.append(- b / (2 * a), " - i", sqrt_val) 
    return roots

In [2]:
# funkce pro zadani polynomu uzivatelem
def enter_polynom():
    while True:
        try:
            level = int(input("Zadejte stupeň polynomu: "))
            assert 0 < level
        except ValueError:
            print("Zadejte číslo.")
        except AssertionError:
            print("Zadejte číslo větší než 0.")
        else:
            break
    i = 0
    p = []
    while i < level + 1:
        while True:
            try:
                koef = int(input(f"Zadejte koeficient x**{level-i}: "))
                p.append(koef)
                i = i + 1
            except ValueError:
                print("Zadejte číslo.")
            else:
                break
    return p

In [11]:
def main():
    roots = []
    err = 1e-9
    max_iter = 10000
    x0 = 2
    p = (1, 0, -5, 0, 4)
    #p = (1, 0, -5, 0, -4)
    #p = (1, 0, -2, 1)
    #p = enter_polynom()
    
    # komplexni koreny
    if isinstance(x0, complex) == True:
        # do pole korenu pridam komplexni a komplexne sdruzeny koreny
        for root in newtons_method(p, x0, max_iter, err):
            roots.append(root)
    else:
        while len(p) > 3:
            # do pole korenu pridavam ty, ktere ziskam newtonovou metodou
            roots.append(newtons_method(p, x0, max_iter, err))

            # polynom kterym budu delit a ktery obsahuje koren (x - koren)
            dividing_p = (1, - roots[len(roots) - 1])
            pn = divide_polynoms(p, dividing_p)

            # na kvadraticky polynom pouziji vzorec
            if len(pn) == 3:
                for root in equation_roots(pn):
                    roots.append(root)
            p = pn
        
    print(f"Kořeny polynomu jsou: {roots}")
    
main()

Kořeny polynomu jsou: [2.0, 1.0, -1.0, -2.0]
